# Задание по лекциям 2.

- Не нужно использовать эффективные вычислительные алгоритмы. Нужно использовать те алгоритмы, что были представлены на лекции или практических занятиях. 
- Остальное как обычно: за "похожие" решения всем задействованным 0 баллов; если используете решение из открытого источника — обязательно вставьте ссылку; не удаляйте формулировки; не выкладывайте в открытый источник.
- Можно использовать `numpy.array` для матриц и матричной арифметики и `numpy.linalg` для подсчёта ранга и определителя, для вычисления обратной матрицы, решения СЛУ и т.п. То есть то, что вы уже реализовывали в прошлом семестре, ещё раз реализовывать необязательно. Более того, можно использовать в любом из *заданий по лекциям* функции, реализованные ранее в других *заданиях по лекциям*. Если возникнут сомнения, можно ли использовать ту или иную функцию — лучше сразу поинтересуйтесь у меня.

In [2]:
import numpy as np
import sympy as sp

$\mathbb{R}^n$ — вещественнозначное пространство вектор-**столбцов** со стандартным скалярным произведением.

**(1 балл) Задание 5.** Реализовать функцию, принимающую на вход набор координат $u = (u_1,\ldots,u_k),\ k\le n$ вектор-столбцов, и выдающую базис $$v = (v_1,\ldots,v_m),\ m \ge n-k,$$
ортогонального дополнения  для линейной оболочки данных векторов.

In [9]:
def basis_orth_comp(U):
    space = sp.Matrix(U.T).nullspace()
    if space:
        basis = sp.Matrix()
        for v in sp.Matrix(U.T).nullspace():
            basis = basis.col_insert(sp.shape(U)[1], v)
            
        return np.array(basis, float)
    
    return np.zeros((U.shape[0], 1))

U = np.array([[1], 
              [1]])
print(basis_orth_comp(U))

U = np.array([[1, 0], 
              [0, 1]]) # orth_comp(V) = {0}, where V = U
print(basis_orth_comp(U))

U = np.array([[1, 0], [0, 1], [0, 0]]) # orth_comp(Oxy) = Oz
print(basis_orth_comp(U))

[[-1.]
 [ 1.]]
[[0.]
 [0.]]
[[0.]
 [0.]
 [1.]]


**(1 балл) Задание 6.** Реализовать функцию, принимающую на вход набор координат $u = (u_1,\ldots,u_k),\ k\le n$ вектор-столбцов, и выдающую ортонормированный базис $$v = (v_1,\ldots,v_m),\ m \ge n-k,$$
для линейной оболочки данных векторов.

In [10]:
def linear_indep_cols(U):
    vs = np.array(sp.Matrix(U.T).echelon_form().T)
    vs = vs[:, np.any(vs != 0, axis=0)]
    return vs.astype(float)
    
def gram_schmidt(U):
    k = U.shape[1]
    for i in range(k):
        u = U[:, i]
        for j in range(i):
            v = U[:, j]
            U[:, i] -= (u @ v) / (v @ v) * v
    return U

def norm(u):
    return u / (u @ u) ** .5

def orth_basis(U):
    bs = linear_indep_cols(U)
    q, r = np.linalg.qr(bs)
    print(q)
    orth_bs = gram_schmidt(bs)
    k = orth_bs.shape[1]
    for i in range(k):
        orth_bs[:, i] = norm(orth_bs[:, i])
    return orth_bs


U = np.array([[1, 2, 3],
              [2, 4, 7], 
              [3, 6, 5]])
basis = orth_basis(U)
print(basis)
print(np.round(basis[:, 0] @ basis[:, 1], 9))
print(basis[:, 0] @ basis[:, 0])
print(basis[:, 1] @ basis[:, 1])

[[-0.26726124 -0.22750788]
 [-0.53452248 -0.77352678]
 [-0.80178373  0.59152048]]
[[ 0.26726124  0.22750788]
 [ 0.53452248  0.77352678]
 [ 0.80178373 -0.59152048]]
0.0
1.0
1.0


**(1 балл) Задание 7.** Реализовать функцию, принимающую на вход набор координат $u = (u_1,\ldots,u_k),\ k\le n$ линейно независимых вектор-столбцов, и выдающую ортонормированный базис $$v = (v_1,\ldots,v_n)$$
всего пространства, такой что линейная оболочка векторов $(v_1,\ldots,v_k)$ совпадает с линейной оболочкой векторов $(u_1,\ldots,u_k)$.

In [11]:
def orth_basis_V(U):
    U = np.float_(U)
    bs = gram_schmidt(U)
    n, k = bs.shape[0], bs.shape[1]
    for i in range(n - k):
        ocomp_bs = basis_orth_comp(bs)
        bs = np.c_[bs, ocomp_bs[:, 0]]
    
    for i in range(n):
        bs[:, i] = norm(bs[:, i])
    return bs


U = np.array([[1],
              [0], 
              [0]])

print(orth_basis_V(U))


U = np.array([[1, 3],
              [2, 7], 
              [3, 5]])

basis_V = orth_basis_V(U)

print(basis_V)
print(np.round(basis_V[:, 0] @ basis_V[:, 1], 9))
print(np.round(basis_V[:, 0] @ basis_V[:, 2], 9))
print(np.round(basis_V[:, 1] @ basis_V[:, 2], 9))
print(np.round(basis_V[:, 0] @ basis_V[:, 0], 9))
print(np.round(basis_V[:, 1] @ basis_V[:, 1], 9))
print(np.round(basis_V[:, 2] @ basis_V[:, 2], 9))

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[ 0.26726124  0.22750788 -0.93638218]
 [ 0.53452248  0.77352678  0.34050261]
 [ 0.80178373 -0.59152048  0.08512565]]
0.0
0.0
0.0
1.0
1.0
1.0
